<h1> Predicting airline delays with Spark and ML-Lib using pySpark </h1>
<br>
Adapted from http://nbviewer.ipython.org/github/ofermend/IPython-notebooks/blob/master/blog-part-2.ipynb

<h3>Pre-processing with PySpark</h3>

In [1]:
import os.path

AIRPORT = 'LAX'

base_dir = os.path.join('data')
input_path_2007 = os.path.join('flights', '2007.csv')
input_path_2008 = os.path.join('flights', '2008.csv')
input_path_LAX = os.path.join('flights', 'LAX.csv')

file_name_2007 = os.path.join(base_dir, input_path_2007)
file_name_2008 = os.path.join(base_dir, input_path_2008)
file_name_LAX = os.path.join(base_dir, input_path_LAX)

raw_data_2007 = sc.textFile(file_name_2007)
raw_data_2008 = sc.textFile(file_name_2008)
weather_data_LAX = sc.textFile(file_name_LAX)

header = raw_data_2007.take(1) 

# filter on Airport
filtered_data_2007 = (raw_data_2007
                        .filter(lambda line: ',' + AIRPORT + ',' in line)
                        # filter out cancelled flights
                        .filter(lambda line: ',,' in line)
                        .filter(lambda line: 'Year' not in line))
filtered_data_2008 = (raw_data_2008
                        .filter(lambda line: ',' + AIRPORT + ',' in line)
                        # filter out cancelled flights
                        .filter(lambda line: ',,' in line)
                        .filter(lambda line: 'Year' not in line))

# CRS = Computer Reservation System
# scheduled time as opposed to the actual time
print header
print filtered_data_2007.take(1)
print filtered_data_2008.take(1)

[u'Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay']
[u'2007,1,1,1,1446,1445,1631,1700,WN,732,N483,225,255,209,-29,1,STL,LAX,1593,7,9,0,,0,0,0,0,0,0']
[u'2008,1,3,4,1738,1715,1838,1820,WN,82,N499WN,60,65,42,18,23,LAS,LAX,236,6,12,0,,0,0,0,0,12,6']


In [2]:
holidays = ['01/01/2007', '01/15/2007', '02/19/2007', '05/28/2007', '06/07/2007', '07/04/2007',
      '09/03/2007', '10/08/2007' ,'11/11/2007', '11/22/2007', '12/25/2007',
      '01/01/2008', '01/21/2008', '02/18/2008', '05/22/2008', '05/26/2008', '07/04/2008',
      '09/01/2008', '10/13/2008' ,'11/11/2008', '11/27/2008', '12/25/2008']

In [3]:
import datetime

def make_weather_dict():
    weather_dict = {}
    weather_lines = weather_data_LAX.collect()
    for line in weather_lines:
        all_vals = line.split(',')
        # key: date, values: wind speed, max temp, min temp, precipitation
        weather_dict[all_vals[2]] = [all_vals[3], all_vals[5], all_vals[6], all_vals[7]]
    
    return weather_dict
    
def days_from_nearest_holiday(year, month, day):
    diffs = []
    sample_date = datetime.date(year, month, day)
    for holiday in holidays:
        dt = datetime.datetime.strptime(holiday, '%m/%d/%Y').date()
        td = dt - sample_date
        diffs.append(abs(td.days))

    return min(diffs) * 1.0

def make_features(flight, weather_dict):
    try:
        flight_data = flight.split(',')
        date_string = str(flight_data[0]) + str(flight_data[1]).zfill(2) + str(flight_data[2]).zfill(2)
        weather_data = weather_dict[date_string]
        features = []
        features.append(float(flight_data[15]))
        features.append(float(flight_data[1]))
        features.append(float(flight_data[2]))
        features.append(float(flight_data[3]))
        features.append(float(flight_data[5]) / 100)
        features.append(float(flight_data[18]))
        features.append(days_from_nearest_holiday(int(flight_data[0]), int(flight_data[1]), int(flight_data[2]))) 
        features.append(float(weather_data[0]))
        features.append(float(weather_data[1]) / 10.0)
        features.append(float(weather_data[2]) / 10.0)
        features.append(float(weather_data[3]))

        return (date_string, features)
    except:
        return (date_string, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [4]:
weather_dict = make_weather_dict()

features_2007 = filtered_data_2007.map(lambda line: make_features(line, weather_dict))
features_2008 = filtered_data_2008.map(lambda line: make_features(line, weather_dict))

print features_2007.take(1)
print features_2008.take(1)

[('20070101', [1.0, 1.0, 1.0, 1.0, 14.45, 1593.0, 0.0, 0.0, 17.2, 7.2, 22.0])]
[('20080103', [23.0, 1.0, 3.0, 4.0, 17.15, 236.0, 2.0, 0.0, 18.3, 12.2, 24.0])]


<h3>Modeling with Spark and ML-Lib</h3>

In [5]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import StandardScaler
import numpy as np

def parse_data(tup):
    if tup[1][0] >= 15:
        return LabeledPoint(1.0, tup[1])
    else:
        return LabeledPoint(0.0, tup[1])
    
def scale(dataset):
    standardizer = StandardScaler(True, True)
    labels = dataset.map(lambda lp: lp.label).collect()
    features = dataset.map(lambda lp: lp.features).collect()
    rdd_features = sc.parallelize(features)
    model = standardizer.fit(rdd_features)
    scaled_features = model.transform(rdd_features).collect()
    zipped = zip(labels, scaled_features)
    labeled_points = []
    for z in zipped:
        labeled_points.append(LabeledPoint(z[0], z[1]))
    return sc.parallelize(labeled_points)

train_data = features_2007.map(parse_data)
train_data.cache()
scaled_train_data = scale(train_data)
scaled_train_data.cache()

test_data = features_2008.map(parse_data)
test_data.cache()
scaled_test_data = scale(test_data)
scaled_test_data.cache()

print scaled_train_data.take(1)
print scaled_test_data.take(1)

[LabeledPoint(0.0, [-0.402295571189,0.0,-1.77624109236,-1.43718831265,0.160527403522,1.6084603387,-1.39015824099,-0.323937227318,-0.125063604608,-0.0661563396444,-0.277943099073])]
[LabeledPoint(1.0, [0.266621329907,0.171080153528,-1.55046145948,0.100742991539,0.782614848046,-0.690575560979,-1.07685279843,-0.547252069547,0.529045541619,1.5462275009,-0.456680001685])]


In [6]:
def eval_metrics(lbl_pred):
    tp = float(lbl_pred.filter(lambda lp: lp[0]==1.0 and lp[1]==1.0).count())
    tn = float(lbl_pred.filter(lambda lp: lp[0]==0.0 and lp[1]==0.0).count())
    fp = float(lbl_pred.filter(lambda lp: lp[0]==1.0 and lp[1]==0.0).count())
    fn = float(lbl_pred.filter(lambda lp: lp[0]==0.0 and lp[1]==1.0).count())
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    F_measure = 2 * precision * recall / (precision + recall)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    return([tp, tn, fp, fn], [precision, recall, F_measure, accuracy])

In [7]:
from pyspark.mllib.classification import LogisticRegressionWithSGD

model_lr = LogisticRegressionWithSGD.train(scaled_train_data, iterations=100)
labels_and_predictions = scaled_test_data.map(lambda lp: (model_lr.predict(lp.features), lp.label))
metrics = eval_metrics(labels_and_predictions)

print('Precision : %.2f' % round(metrics[1][0], 2))
print('Recall : %.2f' % round(metrics[1][1], 2))
print('F1 : %.2f' % round(metrics[1][2], 2))
print('Accuracy : %.2f' % round(metrics[1][3], 2))

Precision : 0.95
Recall : 0.98
F1 : 0.97
Accuracy : 0.98


In [8]:
from pyspark.mllib.classification import SVMWithSGD

model_svm = SVMWithSGD.train(scaled_train_data, iterations=100, step=1.0, regParam=0.01)
labels_and_predictions = scaled_test_data.map(lambda lp: (model_svm.predict(lp.features), lp.label))
metrics = eval_metrics(labels_and_predictions)

print('Precision : %.2f' % round(metrics[1][0], 2))
print('Recall : %.2f' % round(metrics[1][1], 2))
print('F1 : %.2f' % round(metrics[1][2], 2))
print('Accuracy : %.2f' % round(metrics[1][3], 2))

Precision : 0.98
Recall : 0.99
F1 : 0.99
Accuracy : 0.99


In [9]:
from pyspark.mllib.tree import DecisionTree

# Experimental
model_dt = DecisionTree.trainClassifier(scaled_train_data, 2, {}, 'gini', 10, 100)
# this detour is needed because of the spark context error 
# if we map the test data set like in the previous models
list_data = scaled_test_data.collect()
list_predictions = []
for lp in list_data:
    list_predictions.append((model_dt.predict(lp.features), lp.label))

labels_and_predictions = sc.parallelize(list_predictions)
metrics = eval_metrics(labels_and_predictions)

print('Precision : %.2f' % round(metrics[1][0], 2))
print('Recall : %.2f' % round(metrics[1][1], 2))
print('F1 : %.2f' % round(metrics[1][2], 2))
print('Accuracy : %.2f' % round(metrics[1][3], 2))

Precision : 1.00
Recall : 0.70
F1 : 0.82
Accuracy : 0.92
